In [16]:
import os
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from PIL import ImageEnhance, ImageOps
import random
import os
import numpy as np
from PIL import Image
from mtcnn import MTCNN
import cv2
import matplotlib.pyplot as plt
from mtcnn.mtcnn import MTCNN
import os
import random
from tensorflow.keras.models import load_model
import cv2

categories = ["criminal", "non-criminal"]
detector = MTCNN()

Exception ignored in: <_io.BufferedReader>
Traceback (most recent call last):
  File "c:\Users\MuhammadAulia\miniconda3\Lib\site-packages\lz4\frame\__init__.py", line 753, in flush
    self._fp.flush()
ValueError: I/O operation on closed file.
Exception ignored in: <_io.BufferedReader>
Traceback (most recent call last):
  File "c:\Users\MuhammadAulia\miniconda3\Lib\site-packages\lz4\frame\__init__.py", line 753, in flush
    self._fp.flush()
ValueError: I/O operation on closed file.
Exception ignored in: <_io.BufferedReader>
Traceback (most recent call last):
  File "c:\Users\MuhammadAulia\miniconda3\Lib\site-packages\lz4\frame\__init__.py", line 753, in flush
    self._fp.flush()
ValueError: I/O operation on closed file.


In [19]:
loaded_model = load_model("model/criminal_classifier.h5")

def predict_image(img_path, target_size=(64,64), threshold=0.6):
    # Load image
    image_bgr = cv2.imread(img_path)
    if image_bgr is None:
        raise ValueError(f"Gagal load gambar: {img_path}")
    image = cv2.cvtColor(image_bgr, cv2.COLOR_BGR2RGB)

    # Deteksi wajah
    results = detector.detect_faces(image)
    if len(results) != 1:
        print(f"⚠️ {os.path.basename(img_path)} -> {len(results)} wajah terdeteksi, proses testing dilewati.")
        return None

    x, y, w, h = results[0]['box']
    face = image[y:y+h, x:x+w]

    # Resize & grayscale
    face_resized = cv2.resize(face, target_size)
    face_gray = cv2.cvtColor(face_resized, cv2.COLOR_RGB2GRAY)

    # Normalisasi & reshape ke format CNN
    face_norm = face_gray.astype("float32") / 255.0
    face_input = np.expand_dims(face_norm, axis=(0,-1))  # (1,64,64,1)

    # Prediksi
    pred = loaded_model.predict(face_input)
    class_idx = np.argmax(pred)
    confidence = np.max(pred)

    # Jika confidence rendah
    if confidence < threshold:
        print(f"🖼️ File: {os.path.basename(img_path)} | Subject tidak terdeteksi (confidence {confidence:.2f})")

        plt.figure(figsize=(4,4))
        plt.imshow(face_gray, cmap="gray")
        plt.title(f"Subject tidak terdeteksi ({confidence:.2f})")
        plt.axis("off")
        plt.show()
        return "subject tidak terdeteksi", confidence

    # Kalau confidence cukup tinggi → tampilkan label
    print(f"🖼️ File: {os.path.basename(img_path)} | Predicted: {categories[class_idx]} ({confidence:.2f})")

    plt.figure(figsize=(4,4))
    plt.imshow(face_gray, cmap="gray")
    plt.title(f"{categories[class_idx]} ({confidence:.2f})")
    plt.axis("off")
    plt.show()

    return categories[class_idx], confidence


# ==============================
# Coba test dengan satu gambar
# ==============================
# test_img = "data/non-criminal/S18796367/S18796367-GF-A93-EAF-T00-S1-LH-C1-I0035-F00.png"
test_img = "data/tidak-terdeteksi/test2.jpg"
predict_image(test_img, threshold=0.65)

⚠️ test2.jpg -> 2 wajah terdeteksi, proses testing dilewati.
